In [435]:
from function import *
import numpy as np
import networkx as nx
import time
from collections import defaultdict
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import pandas as pd
import matplotlib.pyplot as plt
import warnings

#### LOAD THE DATA

In [436]:
# Using Random Data
# N = 100000                # Number of data
# d = 1                     # Number of features
# precision = 1             # Control precision bit of the output
# dim = d * (2 * precision) # Size of the QUBO matrix
# degree = 2                # Degree of the polynomial regression

# # Generate Random Data
# X = np.random.rand(N, d)
# Y = np.random.rand(N)

# Using Dataset
start_time = time.time()

dataset_path = 'dataset/processed dataset/dataset_8.csv'
dataset_name = dataset_path.split('/')[-1]
df = pd.read_csv(dataset_path)

X = df.iloc[:, :-1].values
Y = df.iloc[:, -1].values
d = X.shape[1]
precision = 2
dim = d * (2 * precision)
degree = 5

# Train-Test data splitting
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# If you want to perform polynomial regression
X_train, d, dim = polynomialForm(X_train, d, dim, precision, degree)
X_test, _, _ = polynomialForm(X_test, d, dim, precision, degree) 

end_time = time.time()
preproccesing1_time = end_time - start_time

print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(1639424, 462)
(409856, 462)
(1639424,)
(409856,)


#### COMPUTING SOME IMPORTANT VARIABLES
Initiating variables that required to creating QUBO Matrix

In [437]:
start_time = time.time()

XtX = np.matmul(X_train.T, X_train) # Covariance matrix
XtY = np.matmul(X_train.T, Y_train) # Relation between Features and Y

end_time = time.time()
preproccesing2_time = end_time - start_time

print(XtX)
print(XtY)

[[1.63942400e+06 1.45892852e+05 9.34460009e+05 ... 1.02709046e+03
  1.58144605e+03 4.41822376e+04]
 [1.45892852e+05 2.37670451e+04 8.15951070e+04 ... 1.43394954e+02
  2.35859400e+02 5.05077740e+03]
 [9.34460009e+05 8.15951070e+04 5.50590568e+05 ... 4.56573040e+02
  7.48054995e+02 2.36264442e+04]
 ...
 [1.02709046e+03 1.43394954e+02 4.56573040e+02 ... 7.24201698e+01
  7.26035646e+01 9.19723965e+01]
 [1.58144605e+03 2.35859400e+02 7.48054995e+02 ... 7.26035646e+01
  9.19723965e+01 1.99439306e+02]
 [4.41822376e+04 5.05077740e+03 2.36264442e+04 ... 9.19723965e+01
  1.99439306e+02 8.94820428e+03]]
[1.50747078e+05 1.65533701e+04 7.93626588e+04 2.82824928e+04
 7.22203982e+03 7.37908317e+03 5.86640456e+04 3.21235197e+03
 8.50275171e+03 3.56749667e+03 1.09164125e+03 1.07054090e+03
 6.49774833e+03 4.36106786e+04 1.38249925e+04 3.19281642e+03
 3.32988431e+03 3.01356774e+04 7.62919241e+03 2.72228997e+03
 2.74329435e+03 1.26721058e+04 3.03471822e+03 4.01140783e+02
 2.88260205e+03 3.45890179e+03 2.8

In [438]:
total_preproccesing_time = preproccesing1_time + preproccesing2_time

#### FIXSTAR AMPLIFY SIMULATED ANNEALER

In [439]:
start_time = time.time()
model = generateQuboMatrix_fixstar(XtX, XtY, dim, precision, d)
end_time = time.time()
fixstar_preproccesing_time = end_time - start_time + total_preproccesing_time

start_time = time.time()
sampleset_fixstar = sampling_fixstar(model)
end_time = time.time()
fixstar_execution_time = end_time - start_time

start_time = time.time()
distributions_fixstar, weights_fixstar = solve_fixstar(sampleset_fixstar, dim, precision, d)
end_time = time.time()
fixstar_postproccesing_time = end_time - start_time

In [440]:
def smape(y_true, y_pred):
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", RuntimeWarning)  
        denominator = np.abs(y_true) + np.abs(y_pred)
        smape = np.where(
            denominator == 0,
            0,
            2 * np.abs(y_true - y_pred) / denominator
        )
    return np.mean(smape) * 100

# Make predictions
Y_train_pred_fixstar = np.dot(X_train, weights_fixstar)
Y_test_pred_fixstar = np.dot(X_test, weights_fixstar)

# Evaluate on training data
train_mse_fixstar = mean_squared_error(Y_train, Y_train_pred_fixstar)
train_mae_fixstar = mean_absolute_error(Y_train, Y_train_pred_fixstar)
train_r2_fixstar = r2_score(Y_train, Y_train_pred_fixstar) 
train_smape_fixstar = smape(Y_train, Y_train_pred_fixstar)

# Evaluate on test data
test_mse_fixstar = mean_squared_error(Y_test, Y_test_pred_fixstar)
test_mae_fixstar = mean_absolute_error(Y_test, Y_test_pred_fixstar)
test_r2_fixstar = r2_score(Y_test, Y_test_pred_fixstar)
test_smape_fixstar = smape(Y_test, Y_test_pred_fixstar)

# Print the metrics
print("Fixstar Training Evaluation:")
print(f"MSE: {train_mse_fixstar:.5f}, MAE: {train_mae_fixstar:.5f}, R²: {train_r2_fixstar:.5f}, SMPAE: {train_smape_fixstar:.5f}%")    

print("\nFixstar Testing Evaluation:")
print(f"MSE: {test_mse_fixstar:.5f}, MAE: {test_mae_fixstar:.5f}, R²: {test_r2_fixstar:.5f}, SMPAE: {test_smape_fixstar:.5f}%")

Fixstar Training Evaluation:
MSE: 0.00008, MAE: 0.00599, R²: 0.99129, SMPAE: 14.44733%

Fixstar Testing Evaluation:
MSE: 0.00008, MAE: 0.00600, R²: 0.99128, SMPAE: 14.44588%


#### NEAL SIMULATED ANNEALER

In [441]:
# start_time = time.time()
# Q = generateQuboMatrix_neal(XtX, XtY, precision, d)
# end_time = time.time()
# neal_preproccesing_time = end_time - start_time + total_preproccesing_time

# start_time = time.time()
# sampleset_neal = sampling_neal(Q)
# end_time = time.time()
# neal_execution_time = end_time - start_time 

# start_time = time.time()
# distributions_neal, weights_neal = solve_neal(sampleset_neal, dim, precision, d)
# end_time = time.time()
# neal_postproccesing_time = end_time - start_time 

In [442]:
# Y_train_pred_neal = np.dot(X_train, weights_neal)
# Y_test_pred_neal = np.dot(X_test, weights_neal)

# # Evaluate on training data
# train_mse_neal = mean_squared_error(Y_train, Y_train_pred_neal)
# train_mae_neal = mean_absolute_error(Y_train, Y_train_pred_neal)
# train_r2_neal = r2_score(Y_train, Y_train_pred_neal)

# # Evaluate on test data
# test_mse_neal = mean_squared_error(Y_test, Y_test_pred_neal)
# test_mae_neal = mean_absolute_error(Y_test, Y_test_pred_neal)
# test_r2_neal = r2_score(Y_test, Y_test_pred_neal)

# # Print the metrics
# print("Neal Training Evaluation:")
# print(f"MSE: {train_mse_neal:.5f}, MAE: {train_mae_neal:.5f}, R²: {train_r2_neal:.5f}")

# print("\nNeal Testing Evaluation:")
# print(f"MSE: {test_mse_neal:.5f}, MAE: {test_mae_neal:.5f}, R²: {test_r2_neal:.5f}")

#### SCIKIT LEARN (TRADISIONAL WAY)

In [443]:
def smape(y_true, y_pred):
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", RuntimeWarning)  
        denominator = np.abs(y_true) + np.abs(y_pred)
        smape = np.where(
            denominator == 0,
            0,
            2 * np.abs(y_true - y_pred) / denominator
        )
    return np.mean(smape) * 100

# Train the model
start_time = time.time() 

clf = LinearRegression()
clf.fit(X_train, Y_train)
print("Coefficients:", clf.coef_)

Y_train_pred_sklearn = clf.predict(X_train)
Y_test_pred_sklearn = clf.predict(X_test)

end_time = time.time()  
sklearn_execution_time = end_time - start_time

# Evaluate on training data
train_mse_sklearn = mean_squared_error(Y_train, Y_train_pred_sklearn)
train_mae_sklearn = mean_absolute_error(Y_train, Y_train_pred_sklearn)
train_r2_sklearn = r2_score(Y_train, Y_train_pred_sklearn) 
train_smape_sklearn = smape(Y_train, Y_train_pred_sklearn)

# Evaluate on test data
test_mse_sklearn = mean_squared_error(Y_test, Y_test_pred_sklearn)
test_mae_sklearn = mean_absolute_error(Y_test, Y_test_pred_sklearn)
test_r2_sklearn = r2_score(Y_test, Y_test_pred_sklearn)
test_smape_sklearn = smape(Y_test, Y_test_pred_sklearn)

# Print the metrics
print("\nSklearn Training Evaluation:")
print(f"MSE: {train_mse_sklearn:.5f}, MAE: {train_mae_sklearn:.5f}, R²: {train_r2_sklearn:.5f}, SMAPE: {train_smape_sklearn:.5f}%")    

print("\nSklearn Testing Evaluation:")
print(f"MSE: {test_mse_sklearn:.5f}, MAE: {test_mae_sklearn:.5f}, R²: {test_r2_sklearn:.5f}, SMAPE: {test_smape_sklearn:.5f}%")

print(f"\nExecution Time: {sklearn_execution_time:.5f} seconds")

Coefficients: [-4.41020740e-13  1.17348012e-02 -3.41005113e-02  8.49110966e-01
 -4.02382303e-02  2.04965523e-02  7.69786424e-02 -3.83842605e-01
  1.14199771e-01 -4.10690697e-02  1.34979967e-01  1.70445305e-01
 -4.75634008e-03  1.56997165e-01  3.36231232e-01  1.55960782e-01
  1.31850801e-01 -2.82912963e-01  1.40504136e+00 -1.35359384e+00
 -1.88403390e+00 -8.54110471e-01 -5.24388826e-01  1.60077415e+00
  3.56182671e-01  2.57856339e-01  4.38525583e-01 -1.36117915e-01
  6.79843455e-01  5.97556428e-02  1.48209962e+00 -5.86938076e-01
  4.19659202e-01  4.95371860e-02 -6.74732434e-01  2.03001680e+00
 -1.18728433e+00 -1.05571287e+00 -5.82483601e-02 -2.98361389e+00
  2.28975411e+00  1.94663885e+00  1.22330785e-01  1.86685986e-03
 -1.24364268e+00 -1.54059514e-01 -9.17855018e-01 -2.50430724e-01
  2.69532681e-01 -3.02417165e-01 -4.07994242e-01 -2.56058996e-01
 -2.63715685e-01  2.73443428e-01 -2.79847454e-01 -2.43841456e+00
  1.74991759e+00  7.30437752e-01  8.42631612e-02 -6.90987278e-01
  6.2576042

#### VISUALIZATION OF GRAPH

In [444]:
# # Initialize the graph and add all nodes first
# G = nx.Graph()
# nodes = set()
# for (i, j) in Q.keys():
#     nodes.add(i)
#     nodes.add(j)
# G.add_nodes_from(nodes)

# # Now add edges and node attributes
# for (i, j), weight in Q.items():
#     if weight != 0 and i <= j:  # Avoid duplicates and zero-weights
#         if i == j:
#             G.nodes[i]['bias'] = weight  # Now safe because node exists
#         else:
#             G.add_edge(i, j, weight=weight)

# # Visualization
# plt.figure(figsize=(12, 10))
# pos = nx.circular_layout(G)

# # Draw nodes and edges
# nx.draw_networkx_nodes(G, pos, node_color='lightblue', node_size=800)
# nx.draw_networkx_labels(G, pos, font_size=12)

# # Scale edge widths by absolute weight
# max_weight = max(abs(d['weight']) for _, _, d in G.edges(data=True))
# edge_widths = [0.1 + 2 * abs(G[u][v]['weight']) / max_weight for u, v in G.edges()]
# nx.draw_networkx_edges(G, pos, width=edge_widths, edge_color='gray')

# # Add edge labels (rounded)
# edge_labels = {(u, v): f"{d['weight']:.1f}" for u, v, d in G.edges(data=True)}
# nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=8)

# # Add node biases
# for node, (x, y) in pos.items():
#     bias = G.nodes[node].get('bias', 0)
#     plt.text(x, y-0.1, f"bias={bias:.1f}", ha='center', fontsize=8, color='red')

# # plt.title("QUBO Graph (Edge Width ∝ |Weight|)", fontsize=14)
# plt.axis('off')
# plt.show()

In [445]:
# pos = nx.spring_layout(G, k=0.5, weight='weight', iterations=100)
# nx.draw(G, pos, with_labels=True, 
#        node_color='lightblue', node_size=800,
#        width=[0.1 + 2*abs(G[u][v]['weight'])/max_weight for u,v in G.edges()],
#        edge_color='gray')

In [446]:
# from mpl_toolkits.mplot3d import Axes3D

# pos = nx.spring_layout(G, dim=3, weight='weight')
# fig = plt.figure()
# ax = fig.add_subplot(111, projection='3d')
# for u, v in G.edges():
#     ax.plot([pos[u][0], pos[v][0]], 
#             [pos[u][1], pos[v][1]], 
#             [pos[u][2], pos[v][2]], 
#             'gray')
# ax.scatter(*np.array(list(pos.values())).T, s=100, c='lightblue')
# plt.show()

#### FINAL EVALUATION

In [447]:
data = {
    "Dataset_Name": [dataset_name] * 4,
    "Number of Row": [df.shape[0]] * 4,  
    "Precision": [precision] * 4,
    "Degree": [degree] * 4,
    "Number_Of_Features": [d] * 4,
    "Model": ["Fixstar", "Fixstar", "Sklearn", "Sklearn"],
    "Dataset_split": ["Train", "Test", "Train", "Test"],
    "MSE": [
        # train_mse_neal, test_mse_neal,
        train_mse_fixstar, test_mse_fixstar,
        train_mse_sklearn, test_mse_sklearn
    ],
    "MAE": [
        # train_mae_neal, test_mae_neal,
        train_mae_fixstar, test_mae_fixstar,
        train_mae_sklearn, test_mae_sklearn
    ],
    "R2 Score": [
        # train_r2_neal, test_r2_neal,
        train_r2_fixstar, test_r2_fixstar,
        train_r2_sklearn, test_r2_sklearn
    ],
    "SMAPE": [
        # train_mape_neal, test_mape_neal,
        train_smape_fixstar, test_smape_fixstar,
        train_smape_sklearn, test_smape_sklearn
    ],
    "preproccesing_time": [
        # neal_preproccesing_time, neal_preproccesing_time,
        fixstar_preproccesing_time, fixstar_preproccesing_time,
        "-", "-" 
    ],
    "execution_time": [
        # neal_execution_time, neal_execution_time,
        fixstar_execution_time, fixstar_execution_time,
        sklearn_execution_time, sklearn_execution_time
    ],
    "postproccesing_time": [
        # neal_postproccesing_time, neal_postproccesing_time,
        fixstar_postproccesing_time, fixstar_postproccesing_time,
        "-", "-"
    ],
    "total_time": [
        # neal_preproccesing_time + neal_execution_time + neal_postproccesing_time,neal_preproccesing_time + neal_execution_time + neal_postproccesing_time,
        fixstar_preproccesing_time + fixstar_execution_time + fixstar_postproccesing_time,fixstar_preproccesing_time + fixstar_execution_time + fixstar_postproccesing_time,
        sklearn_execution_time, sklearn_execution_time
    ]
}

df_eval = pd.DataFrame(data)
df_eval

,Dataset_Name,Number of Row,Precision,Degree,Number_Of_Features,Model,Dataset_split,MSE,MAE,R2 Score,SMAPE,preproccesing_time,execution_time,postproccesing_time,total_time
0,dataset_8.csv,2049280,2,5,462,Fixstar,Train,0.000080,0.005989,0.991292,14.447331,32.970771,39.217940,0.004463,72.193174
1,dataset_8.csv,2049280,2,5,462,Fixstar,Test,0.000080,0.005997,0.991278,14.445881,32.970771,39.217940,0.004463,72.193174
2,dataset_8.csv,2049280,2,5,462,Sklearn,Train,0.000009,0.001800,0.999040,6.028618,-,168.189975,-,168.189975
3,dataset_8.csv,2049280,2,5,462,Sklearn,Test,0.000009,0.001798,0.999055,6.002249,-,168.189975,-,168.189975


In [448]:
excel_filename = "evaluation_results3.xlsx"

# Add new data
append_to_excel_enhanced(
    excel_filename,
    df_eval,
    check_columns=['Dataset_Name', 'Precision', 'Degree']
)

Data successfully appended to Excel file.
